The goal here is to show how we can implement Euler's method using Julia! 
Let's consider a disability insurance, this means that $S = \{*, \diamond, \dagger\}$, we want to find the survivalprobabilities

$$\begin{aligned}
P(t,s) &= 
\begin{bmatrix}
    p_{**}(t,s)         & p_{*\diamond}(t,s)        & p_{*\dagger}(t,s) \\
    p_{\diamond*}(t,s)  & p_{\diamond\diamond}(t,s) & p_{\diamond\dagger}(t,s) \\
    0                   &  0                        & 1
\end{bmatrix}
\end{aligned}$$

In [3]:
using LinearAlgebra

In [12]:
#states: 
# 0: alive, 1: disabeld, 2: deceased

# Transition rate matrix Λ
function Λ(t)
    #state0:
    μ01(t) = 0.0004 + 10^(0.06*t-5.46)
    μ02(t) = 0.0005 + 10^(0.038*t-4.12)
    μ00(t) = -(μ01(t) + μ02(t))
    #state1:
    μ10(t) = 0.05 
    μ12(t) = μ02(t)
    μ11(t) = -(μ10(t)+μ12(t))
    #state2:
    # transition rates in the deceased state are zero
    
    L = [μ00(t) μ01(t) μ02(t)
         μ10(t) μ11(t) μ12(t)
         0       0     0    ]
    
    return L
end


Λ (generic function with 1 method)

In [13]:
function f(t,M)
    return M*Λ(t)
end

function Euler(t0, P0, h, tn)
    if t0 == tn
        return P0
    end

    N = Int(round((tn-t0)/h))
    D = Int(size(P0)[1])
    # store N+1 (DxD)-matricies
    P = zeros(Float64,D,D,N+1) 
    P[:,:,1] = P0 

    
    for n in 1:N
        P[:,:, n+1] = P[:,:,n] + h*f(t0+n*h, P[:,:,n])
    end

    return P
end 

Euler (generic function with 1 method)

In [14]:
t0 = 30 
P0 = Matrix(1.0*I, 3,3)
h = 1/(365)  #daily stepsize 
tn = 120

sol = Euler(t0, P0, h, tn) # transition probabilities from 30 to 120 with stepsize h =1/12
print(sol[1 , 1, 1:10])    # first 10 steps of survival


[1.0, 0.9999940650997714, 0.9999881295520607, 0.9999821933566287, 0.9999762565132362, 0.9999703190216436, 0.9999643808816115, 0.9999584420929007, 0.9999525026552715, 0.9999465625684842]